In [ ]:
!nvidia-smi

In [1]:
#config consts
DATASET =     {'train':'train.tar.gz',
               'test':'test.tar.gz',
               'train_mask':'train_mask.tar.gz'}

METADATA=     {'image_level':'train_image_level.csv',
               'study_level':'train_study_level.csv',
               'df_meta':'df_meta.csv',
               'df_fold_rand830':'df_fold_rand830.csv',
               'train_dup':'duplicate.txt',
               'sample_sub':'sample_submission.csv'}

CFGMODEL_DIR_DICT = {'B3_512':'effb3-full-512-mask-v8/',
                     'B3_512_PNG': 'effb3-512-png-mask/',
                     'B4_512':'effb4-full-512-mask/',
                     'B5_600':'effb5-600-mask/',
                     'B5_640':'effb5-640-mask/',
                     'B5_640_M2': 'effb5-640-mask/',
                     'D121_640':'d121-640-mask/',
                     'D201_640':'d201-640-mask/',
                     'B7_768':'effb7-768-mask/',
                     'MLP_640':'mlp-640/',}

INPUT_DIR ='/content/drive/My\ Drive/kaggle/covid19-det/input/'

OUTPUT_DIR = {'BSL':'/content/drive/My\ Drive/kaggle/covid19-det/output/',
              'NORM':'/content/drive/My Drive/kaggle/covid19-det/output/'}

IMPORT_DIR = '/content/drive/My Drive/kaggle/covid19-det/nbs/py/'

HENGCK_IM_DIR=IMPORT_DIR+'hengck_code/dummy_01q/'

WORK_DIR='/content/'

DATASET_DIR_DICT = {'256': INPUT_DIR+'256_jpg/',
                    '512': INPUT_DIR+'512_jpg/',
                    '512_PNG': INPUT_DIR+'512_png/',
                    '600': INPUT_DIR+'600_jpg/',
                    '640': INPUT_DIR+'640_jpg/',
                    '640_M2':INPUT_DIR+'640_jpg_m2/',
                    '768':INPUT_DIR+'768_jpg/'}

EXPERIMENT='SZ768_B7_NEWTEST'
EXPERIMENT_DIR = OUTPUT_DIR['BSL'] + EXPERIMENT+'/'
CFGMODEL_DIR = CFGMODEL_DIR_DICT['B7_768']
MASK_SIZE=(48,48)
L1_FACTOR=4
LR_FACTOR=0.1
MAX_LR_PLATEAU=20
PROBS=[0.2,0.2] #mixup, cutmix 
DATASET_DIR = DATASET_DIR_DICT['768']
METADATA_DIR = INPUT_DIR+'metadata/'
FOLDS_SET=[0]
INITIAL_CHECKPOINTS=[None,None,None,None,None]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydicom
!pip install madgrad
!pip install timm

import sys
sys.path.append(HENGCK_IM_DIR)
sys.path.append(HENGCK_IM_DIR+CFGMODEL_DIR)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from common import *

from siim import *

# from lib.net.lookahead import *
# from lib.net.radam import *
from madgrad import MADGRAD

from model import *
from dataset import *


def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print('mkdir',work_dir+record)
    !mkdir {work_dir+record}
    print ('tar -xzf',work_dir+ds_dict[record],'-C',work_dir+record)
    !tar -xzf  {work_dir+ds_dict[record]} -C {work_dir+record}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}
copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
!ls /content/
!mkdir {EXPERIMENT_DIR}

     |████████████████████████████████| 376 kB 5.0 MB/s 
copy /content/drive/My\ Drive/kaggle/covid19-det/input/768_jpg/train.tar.gz  to /content/
mkdir /content/train
tar -xzf /content/train.tar.gz -C /content/train
rm  /content/train.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/768_jpg/test.tar.gz  to /content/
mkdir /content/test
tar -xzf /content/test.tar.gz -C /content/test
rm  /content/test.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/768_jpg/train_mask.tar.gz  to /content/
mkdir /content/train_mask
tar -xzf /content/train_mask.tar.gz -C /content/train_mask
rm  /content/train_mask.tar.gz
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_image_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_study_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_meta.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_f

In [ ]:
#----------------
import torch.cuda.amp as amp

class AmpNet(Net):
    @torch.cuda.amp.autocast()
    def forward(self,*args):
        return super(AmpNet, self).forward(*args)

is_mixed_precision = True  #True #False
#run_check_net()

def rand_bboxes(size, gamma):
    W = size[0]
    H = size[1]
    GS = gamma.shape[0]
    cut_rat = np.sqrt(1. - gamma)
    cut_w = np.int_(W * cut_rat)
    cut_h = np.int_(H * cut_rat)

    # uniform
    cx = np.random.randint(W,size=GS)
    cy = np.random.randint(H,size=GS)


    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
 
    return bbx1, bby1, bbx2, bby2

def do_mixup(batch1,batch2,alpha=0.4):
  bs = len(batch1['index'])
  gamma = np.random.beta(alpha, alpha, bs)
  gamma = torch.FloatTensor(np.max(np.stack((gamma,1-gamma)),axis=0))
  gamma_4 = gamma[:,None,None,None].cuda()
  gamma_2 = gamma[:,None].cuda()

  mix_image = gamma_4*batch1['image'].cuda()+(1-gamma_4)*batch2['image'].cuda()
  mix_mask = gamma_4*batch1['mask'].cuda()+(1-gamma_4)*batch2['mask'].cuda()
  mix_onehot = gamma_2*batch1['onehot'].cuda()+(1-gamma_2)*batch2['onehot'].cuda()
  
 
  mix_batch={'image':mix_image,
             'mask': mix_mask,
             'onehot':mix_onehot,
             'gamma': gamma}

  return mix_batch

def do_cutmix(batch1, batch2,alpha=0.4):
    bs = len(batch1['index'])
    gamma = np.random.beta(alpha, alpha,bs)
    gamma = torch.FloatTensor(np.max(np.stack((gamma,1-gamma)),axis=0))
    bbx1, bby1, bbx2, bby2 = rand_bboxes((image_size,image_size), gamma.numpy())

    cutmix_image = batch1['image'].detach().clone().cuda()
    cutmix_mask = batch1['mask'].detach().clone().cuda()
    for i in range(bs):
      cutmix_image[i, :, bbx1[i]:bbx2[i], bby1[i]:bby2[i]] = batch2['image'][i, :, bbx1[i]:bbx2[i], bby1[i]:bby2[i]]
      cutmix_mask[i, :, bbx1[i]:bbx2[i], bby1[i]:bby2[i]] = batch2['mask'][i, :, bbx1[i]:bbx2[i], bby1[i]:bby2[i]]

    # adjust gamma to exactly match pixel ratio
    gamma = torch.FloatTensor(1 - ((bbx2 - bbx1) * (bby2 - bby1) / (image_size*image_size)))
    gamma_2 = gamma[:,None].cuda()
    
    
    cutmix_onehot = gamma_2*batch1['onehot'].cuda()+(1-gamma_2)*batch2['onehot'].cuda()
    

    cutmix_batch = {'image': cutmix_image,
                    'mask': cutmix_mask,
                    'onehot': cutmix_onehot,
                    'gamma': gamma}

    
    return cutmix_batch
  
def draw_batch(imbatch):
  bs = imbatch.shape[0]
  fig, axs = plt.subplots(1, bs, figsize=(30, 30))
  for i in range(bs):
    axs[i].imshow(imbatch[i,0,:,:],cmap='gray')
  plt.show()

def train_augment(r):
    image = r['image']
    mask = r['mask']
    # if image[:2].shape != (image_size, image_size):
    #     image = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_AREA)

    if 1:
        for fn in np.random.choice([
            lambda image, mask : do_random_scale(image, mask, mag=0.20),
            lambda image, mask : do_random_stretch_y(image, mask, mag=0.20),
            lambda image, mask : do_random_stretch_x(image, mask, mag=0.20),
            lambda image, mask : do_random_shift(image, mask, mag=int(0.20*image_size)),
            lambda image, mask : (image, mask)
        ],1):
            image, mask = fn(image, mask)

        for fn in np.random.choice([
            lambda image, mask : do_random_rotate(image, mask, mag=15),
            lambda image, mask : do_random_hflip(image, mask),
            lambda image, mask : (image, mask)
        ],1):
            image, mask = fn(image, mask)

        # ------------------------
        for fn in np.random.choice([
            lambda image : do_random_intensity_shift_contast(image, mag=[0.5,0.5]),
            lambda image : do_random_noise(image, mag=0.05),
            lambda image : do_random_guassian_blur(image),
            lambda image : do_random_blurout(image, size=0.25, num_cut=2),
            #lambda image : do_random_clahe(image),
            #lambda image : do_histogram_norm(image),
            lambda image : image,
        ],1):
            image = fn(image)

    r['image'] = image
    r['mask'] = mask
    return r
#----------------


def do_valid(net, valid_loader):

    valid_probability = []
    valid_truth = []
    valid_num = 0

    net.eval()
    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        image = batch['image'].cuda()
        onehot = batch['onehot']
        label = onehot.argmax(-1)

        with torch.no_grad():
            #with amp.autocast():
                logit,mask = data_parallel(net,image)
                probability = F.softmax(logit,-1)

        valid_num += batch_size
        valid_probability.append(probability.data.cpu().numpy())
        valid_truth.append(label.data.cpu().numpy())
        print('\r %8d / %d  %s'%(valid_num, len(valid_loader.dataset),time_to_str(timer() - start_timer,'sec')),end='',flush=True)

    assert(valid_num == len(valid_loader.dataset))
    #print('')
    #----------------------
    truth = np.concatenate(valid_truth)
    probability = np.concatenate(valid_probability)
    predict = probability.argsort(-1)[:,::-1]

    loss = np_loss_cross_entropy(probability,truth)
    topk = (predict==truth.reshape(-1,1))
    acc  = topk[:, 0]
    topk = topk.mean(0).cumsum()
    acc = [acc[truth==i].mean() for i in range(num_study_label)]

    #---
    map  = np_metric_map_curve_by_class(probability, truth)*(4/6)

    return [loss, map.mean(), topk[0], topk[1]]

In [ ]:
run_check_net()

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b7_ra-6c08e654.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b7_ra-6c08e654.pth


torch.Size([2, 3, 768, 768])
torch.Size([2, 4])
torch.Size([2, 1, 48, 48])


In [ ]:
FOLDS_SET=[0]
INITIALINITIAL_CHECKPOINTS=[None,None,None,None,None]

In [ ]:
def run_train_mixup_cutmix():
    for fold in FOLDS_SET:
        out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold
        
        initial_checkpoint=None
        if INITIAL_CHECKPOINTS[fold] is not None: 
          initial_checkpoint = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d/checkpoint/'%fold+INITIAL_CHECKPOINTS[fold]

        best_map = 0.3
        start_lr   = 0.001#1
        min_lr =     0.000005
        batch_size = 4 #14 #22

        num_iteration = 25000
        iter_log    = 200
        iter_valid  = 200
        iter_save   = list(range(0, num_iteration+1, 200))
        a_iter_save = []

        ## setup  ----------------------------------------
        for f in ['checkpoint', 'train', 'valid', 'backup']: os.makedirs(out_dir + '/' + f, exist_ok=True)
        # backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.train.%s.zip'%IDENTIFIER)

        log = Logger()
        log.open(out_dir + '/log.train.txt', mode='a')
        log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
        log.write('\t%s\n' % COMMON_STRING)
        log.write('\texpirement = %s\n' % EXPERIMENT)
        log.write('\tout_dir  = %s\n' % out_dir)
        log.write('\n')

        ## dataset ------------------------------------
        df_train, df_valid = make_fold('train-%d'%fold)
        train_dataset = SiimDataset(df_train, train_augment)
        valid_dataset = SiimDataset(df_valid, )

        train_loader1 = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size,
            drop_last   = True,
            num_workers = 2,
            pin_memory  = True,
            worker_init_fn=lambda id: np.random.seed(torch.initial_seed() // 2 ** 32 + id),
            collate_fn  = null_collate,
        )
        train_loader2 = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size,
            drop_last   = True,
            num_workers = 2,
            pin_memory  = True,
            worker_init_fn=lambda id: np.random.seed(torch.initial_seed() // 2 ** 32 + id),
            collate_fn  = null_collate,
        )
        valid_loader  = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size  = batch_size,
            drop_last   = False,
            num_workers = 2,
            pin_memory  = True,
            collate_fn  = null_collate,
        )

        log.write('train_dataset : \n%s\n'%(train_dataset))
        log.write('valid_dataset : \n%s\n'%(valid_dataset))
        log.write('\n')


        ## net ----------------------------------------
        log.write('** net setting **\n')
        if is_mixed_precision:
            scaler = amp.GradScaler()
            net = AmpNet().cuda()
        else:
            net = Net().cuda()


        if initial_checkpoint is not None:
            f = torch.load(initial_checkpoint, map_location=lambda storage, loc: storage)
            start_iteration = f['iteration']
            num_iteration = start_iteration + 15000
            start_epoch = f['epoch']
            state_dict  = f['state_dict']
            start_lr = f['lrate']
            best_map = f['map']
            net.load_state_dict(state_dict,strict=True)  #True
        else:
            start_iteration = 0
            start_epoch = 0


        log.write('net=%s\n'%(type(net)))
        log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
        log.write('\n')

        # -----------------------------------------------
        if 0: ##freeze
            for p in net.block0.backbone.parameters(): p.requires_grad = False


        #optimizer = Lookahead(RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr), alpha=0.5, k=5)
        #optimizer = RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr)
        optimizer = MADGRAD( filter(lambda p: p.requires_grad, net.parameters()), lr=start_lr, momentum= 0.9, weight_decay= 0, eps= 1e-06)



        log.write('optimizer\n  %s\n'%(optimizer))
        log.write('\n')


        ## start training here! ##############################################
        log.write('** start training here! **\n')
        log.write('   fold = %d\n'%(fold))
        log.write('   is_mixed_precision = %s \n'%str(is_mixed_precision))
        log.write('   batch_size = %d\n'%(batch_size))
        log.write('   experiment = %s\n' % str(EXPERIMENT.split('/')[-2:]))
        log.write('                             |-----------VALID-------------|-----TRAIN/BATCH -----|------TIME-----\n')
        log.write('rate  plateau   iter   epoch | loss    map   topk0  topk1  | loss0  loss1  loss2  |               \n')
        log.write('--------------------------------------------------------------------------------------------------\n')
                  #0.00000    00   0.00*   0.00 | 0.000  0.000  0.000  0.000  | 0.000  0.000  0.000  |  0 hr 00 min

        def message(mode='print'):
            if mode==('print'):
                asterisk = ' '
                loss = batch_loss
            if mode==('log'):
                asterisk = '*' if iteration in a_iter_save else ' '
                loss = train_loss

            text = \
                '%0.5f  %-7d %5.3f%s %4.2f  | '%(rate,rate_plateau, iteration/10000, asterisk, epoch,) +\
                '%4.3f  %4.3f  %4.3f  %4.3f  | '%(*valid_loss,) +\
                '%4.3f  %4.3f  %4.3f  | '%(*loss,) +\
                '%s' % (time_to_str(timer() - start_timer,'min'))

            return text

        #----
        valid_loss = np.zeros(4,np.float32)
        train_loss = np.zeros(3,np.float32)
        batch_loss = np.zeros_like(train_loss)
        sum_train_loss = np.zeros_like(train_loss)
        sum_train = 0
        loss0 = torch.FloatTensor([0]).cuda().sum()
        loss1 = torch.FloatTensor([0]).cuda().sum()
        loss2 = torch.FloatTensor([0]).cuda().sum()


        start_timer = timer()
        iteration = start_iteration
        epoch = start_epoch
        rate = start_lr
        rate_plateau = 0

        while  iteration < num_iteration:

            if (rate<min_lr): break
            it_loader2 = iter(train_loader2)

            for t, batch1 in enumerate(train_loader1):

                if (iteration % iter_valid == 0):

                        valid_loss = do_valid(net, valid_loader)

                        if best_map < valid_loss[1]:

                          best_map = valid_loss[1]
                          a_iter_save.append(iteration)
                          rate_plateau=0
                          torch.save({
                            'state_dict': net.state_dict(),
                            'iteration': iteration,
                            'epoch': epoch,
                            'lrate': rate,
                            'map':best_map}, out_dir + '/checkpoint/best_model.pth')
                          
                        else:
                          rate_plateau=rate_plateau+1


                if (iteration % iter_log == 0):
                  print('\r', end='', flush=True)
                  log.write(message(mode='log') + '\n')


                # learning rate schduler ------------
                rate = get_learning_rate(optimizer)

                if (rate_plateau>MAX_LR_PLATEAU):
                  rate_plateau=0
                  rate = rate*LR_FACTOR
                  best_model_pth = out_dir+'/checkpoint/best_model.pth'
                  f = torch.load(best_model_pth, map_location=lambda storage, loc: storage)
                  state_dict  = f['state_dict']
                  iteration = f['iteration']
                  epoch = f['epoch']
                  net.load_state_dict(state_dict,strict=True)
                  del optimizer
                  optimizer = MADGRAD( filter(lambda p: p.requires_grad, net.parameters()), lr=rate, momentum= 0.9, weight_decay= 0, eps= 1e-06)
                  break

                # mixup cutmix--------------
              
                with torch.no_grad():
                  prob = np.random.rand()
                  if prob<PROBS[0]:
                    batch2 = next(it_loader2)
                    mix_batch = do_mixup(batch1,batch2)
                  elif prob>=PROBS[0] and prob<PROBS[0]+PROBS[1]:
                    batch2 = next(it_loader2)
                    mix_batch = do_cutmix(batch1,batch2)
                  else:
                    mix_batch = {'image' :batch1['image'].cuda(),
                                 'mask'  :batch1['mask'].cuda(),
                                 'onehot':batch1['onehot'].cuda()}


                # one iteration update  -------------
                batch_size = len(batch1['index'])
                image = mix_batch['image']
                truth_mask = mix_batch['mask']
                truth_mask = F.interpolate(truth_mask, size=MASK_SIZE, mode='bilinear', align_corners=False)
                onehot = mix_batch['onehot']
                label = onehot.argmax(-1)

                #----
                net.train()
                optimizer.zero_grad()

                if is_mixed_precision:
                    with amp.autocast():
                        logit,mask = data_parallel(net, image)
                        loss0 = F.cross_entropy(logit, label)
                        loss1 = L1_FACTOR*F.binary_cross_entropy_with_logits(mask, truth_mask)
                        loss2 = loss0 + loss1

                    #scaler.scale(loss0).backward()
                    #scaler.scale(loss1).backward()
                    scaler.scale(loss0+loss1).backward()
                    scaler.unscale_(optimizer)
                    #torch.nn.utils.clip_grad_norm_(net.parameters(), 2)
                    scaler.step(optimizer)
                    scaler.update()


                else :
                    assert(False)
                    print('fp32')
                    logit, mask = data_parallel(net, image)
                    loss0 = F.cross_entropy(logit, label)
                    loss1 = F.binary_cross_entropy_with_logits(mask, truth_mask.shape)

                    (loss0+loss1).backward()
                    optimizer.step()


                # print statistics  --------
                epoch += 1 / len(train_loader1)
                iteration += 1

                batch_loss = np.array([loss0.item(), loss1.item(), loss2.item()])
                sum_train_loss += batch_loss
                sum_train += 1
                if iteration % 100 == 0:
                    train_loss = sum_train_loss / (sum_train + 1e-12)
                    sum_train_loss[...] = 0
                    sum_train = 0

                print('\r', end='', flush=True)
                print(message(mode='print'), end='', flush=True)


        log.write('\n')

In [ ]:
run_train_mixup_cutmix()


--- [START 2021-08-08_13-36-33] ----------------------------------------------------------------

	@common.py:  
	pytorch
		seed = 1628429798
		torch.__version__              = 1.9.0+cu102
		torch.version.cuda             = 10.2
		torch.backends.cudnn.version() = 7605
		os['CUDA_VISIBLE_DEVICES']     = 0
		torch.cuda.device_count()      = 1
		torch.cuda.get_device_properties() = (name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


	expirement = SZ768_B7_NEWTEST
	out_dir  = /content/drive/My Drive/kaggle/covid19-det/output/SZ768_B7_NEWTEST/fold0

train_dataset : 
	len = 4980
	df  = (4980, 10)
	label distribution
		 0     Negative for Pneumonia:  1366 (0.2743)
		 1         Typical Appearance:  2367 (0.4753)
		 2   Indeterminate Appearance:   864 (0.1735)
		 3        Atypical Appearance:   383 (0.0769)

valid_dataset : 
	len = 1247
	df  = (1247, 10)
	label distribution
		 0     Negative for Pneumonia:   343 (0.2751)
		 1         Typical Appear